In [1]:
library(Seurat)
library(SeuratDisk)
library(caret)

Attaching SeuratObject

Registered S3 method overwritten by 'cli':
  method     from         
  print.boxx spatstat.geom

Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat

Loading required package: ggplot2

Loading required package: lattice



In [2]:
# library(SeuratDisk)
# Convert ( "../scClass_data/cordblood_pp_match.h5ad" , dest  =  "h5seurat" , overwrite  =  FALSE )
# Convert ("../scClass_data/pbmc68k_v2.h5ad", dest="h5seurat", overwrite=TRUE) #if the file exist, then overwrite

In [3]:
ref_file <- "../scClass_data/pbmcmm_pp_subsample.h5seurat"
test.file <- "../scClass_data/cordblood_pp_match.h5seurat"

In [4]:
ref <- LoadH5Seurat(ref_file)

Validating h5Seurat file

Initializing RNA with data

Adding counts for RNA

Adding feature-level metadata for RNA

Adding command information

Adding cell-level metadata

Adding miscellaneous information

Adding tool-specific results



In [5]:
ref[["random"]] <- sample(0:1, 16778, replace=TRUE)
ref.list <- SplitObject(ref, split.by = "random")
ref.list

$`1`
An object of class Seurat 
20729 features across 8418 samples within 1 assay 
Active assay: RNA (20729 features, 0 variable features)

$`0`
An object of class Seurat 
20729 features across 8360 samples within 1 assay 
Active assay: RNA (20729 features, 0 variable features)


In [6]:
for (i in 1:length(ref.list)) {
    ref.list[[i]] <- NormalizeData(ref.list[[i]], verbose = FALSE)
    ref.list[[i]] <- FindVariableFeatures(ref.list[[i]], selection.method = "vst", nfeatures = 2000,verbose = FALSE)
}

In [7]:
ref.anchors <- FindIntegrationAnchors(object.list = ref.list, dims = 1:30)
ref.integrated <- IntegrateData(anchorset = ref.anchors, dims = 1:30)
ref.integrated <- ScaleData(ref.integrated, verbose = FALSE)
ref.integrated <- RunPCA(ref.integrated, npcs = 30, verbose = FALSE)
ref.integrated <- RunUMAP(ref.integrated, reduction = "pca", dims = 1:30, verbose = FALSE) # ,return.model = TRUE

Computing 2000 integration features

Scaling features for provided objects

Finding all pairwise anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 20451 anchors

Filtering anchors

	Retained 15375 anchors

Merging dataset 2 into 1

Extracting anchors for merged samples

Finding integration vectors

Finding integration vector weights

Integrating data

Warning message:
“The default method for RunUMAP has changed from calling Python UMAP via reticulate to the R-native UWOT using the cosine metric
To use Python UMAP via reticulate, set umap.method to 'umap-learn' and metric to 'correlation'
This message will be shown once per session”


## Cell type classification

In [8]:
test.dataset <- LoadH5Seurat(test.file)

Validating h5Seurat file

Initializing RNA with data

Adding counts for RNA

Adding feature-level metadata for RNA

Adding command information

Adding cell-level metadata

Adding miscellaneous information

Adding tool-specific results



In [9]:
ref.anchors <- FindTransferAnchors(reference = ref.integrated, query =test.dataset,dims = 1:30, reference.reduction = "pca")

Projecting cell embeddings

Finding neighborhoods

Finding anchors

	Found 6174 anchors

Filtering anchors

	Retained 542 anchors



In [12]:
ref.integrated$transfer_cell_type <- as.character(ref.integrated$transfer_id) # celltype = transfer_id (int to char)
predictions <- TransferData(anchorset = ref.anchors, refdata = ref.integrated$transfer_cell_type,dims = 1:30)
test.dataset <- AddMetaData(test.dataset, metadata = predictions)

Finding integration vectors

Finding integration vector weights

Predicting cell labels



In [11]:
predictions

ERROR: Error in eval(expr, envir, enclos): object 'predictions' not found


## Make Confusion Matrix

In [20]:
label <- c('helper T cell','cytotoxic T cell','memory B cell','naive B cell',
           'plasma cell','natural killer cell','erythrocyte',
           'megakaryocyte','monocyte','dendritic cell','HSPC')

In [21]:
y.pred <- factor(label[as.integer(test.dataset$predicted.id)+1],levels=label)
y.true <- factor(test.dataset$transfer_cell_type,levels=label)

In [41]:
y.pred <- y.pred[!is.na(y.true)]
y.true <- y.true[!is.na(y.true)]

In [42]:
mat.table <- confusionMatrix(y.true,y.pred)
mat.df <- as.data.frame.matrix(mat.table$table) 
colnames(mat.df) = 0:10
mat.df

0 1     2 3     4 5   6    7   8    9   10 
helper T cell       0 30266 0     7 0  27   37   0   11   0   0
cytotoxic T cell    0 12975 0     1 0  13   13   0    0   0   0
memory B cell       0    87 0   649 0   2    0   0    1   1   0
naive B cell        0    10 0 14484 0   0    4   0    0   0   0
plasma cell         0     0 0     0 0   0    0   0    0   0   0
natural killer cell 0  4788 0    53 0 746   14   0    3   0   2
erythrocyte         0    79 0    67 0  19 1137   8    2   3 934
megakaryocyte       0   474 0    74 0  11    2 303    7   3  22
monocyte            0   531 0    23 0   0    9   4 7410 949   1
dendritic cell      0     3 0    15 0   0    0   0    3 590   0
HSPC                0     0 0     0 0   0    0   0    0   0   0

In [43]:
mat.df.percent <- round(mat.df/(rowSums(mat.df)+0.001)*100, digits=1)
mat.df.percent

0 1    2 3    4 5    6    7    8    9    10  
helper T cell       0 99.7 0  0.0 0  0.1  0.1  0.0  0.0  0.0  0.0
cytotoxic T cell    0 99.8 0  0.0 0  0.1  0.1  0.0  0.0  0.0  0.0
memory B cell       0 11.8 0 87.7 0  0.3  0.0  0.0  0.1  0.1  0.0
naive B cell        0  0.1 0 99.9 0  0.0  0.0  0.0  0.0  0.0  0.0
plasma cell         0  0.0 0  0.0 0  0.0  0.0  0.0  0.0  0.0  0.0
natural killer cell 0 85.4 0  0.9 0 13.3  0.2  0.0  0.1  0.0  0.0
erythrocyte         0  3.5 0  3.0 0  0.8 50.6  0.4  0.1  0.1 41.5
megakaryocyte       0 52.9 0  8.3 0  1.2  0.2 33.8  0.8  0.3  2.5
monocyte            0  5.9 0  0.3 0  0.0  0.1  0.0 83.0 10.6  0.0
dendritic cell      0  0.5 0  2.5 0  0.0  0.0  0.0  0.5 96.6  0.0
HSPC                0  0.0 0  0.0 0  0.0  0.0  0.0  0.0  0.0  0.0

In [44]:
print(paste('acc:',sum(y.pred==y.true)/length(y.pred)*100,'%'))

[1] "acc: 48.967831731207 %"
